# Distribution Generation Curtailment with OPF
This is an introduction on how to use the pandapower optimal power flow for calculation optimal distributed generation curtailment.

## Example Network

We use the four bus example network from the basic OPF tutorial:

<img src="pics/example_opf.png" width="50%">

We first create this network in pandapower:

In [ ]:
import pandas as pd
from pandapower.create import (
    create_empty_network,
    create_ext_grid,
    create_bus,
    create_transformer,
    create_load,
    create_lines,
    create_gen,
    create_poly_cost
)
from pandapower.run import runopp

In [ ]:
net = create_empty_network()

#create buses
bus1 = create_bus(net, vn_kv=220., min_vm_pu=1.0, max_vm_pu=1.02)
bus2 = create_bus(net, vn_kv=110., min_vm_pu=1.0, max_vm_pu=1.02)
bus3 = create_bus(net, vn_kv=110., min_vm_pu=1.0, max_vm_pu=1.02)
bus4 = create_bus(net, vn_kv=110., min_vm_pu=1.0, max_vm_pu=1.02)

#create 220/110 kV transformer
create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV", max_loading_percent=100)

#create 110 kV lines
create_lines(net, [bus2, bus3, bus4], [bus3, bus4, bus2], length_km=[70, 50, 40], std_type='149-AL1/24-ST1A 110.0', max_loading_percent=100)

#create loads
create_load(net, bus2, p_mw=60, controllable=False)
create_load(net, bus3, p_mw=70, controllable=False)
create_load(net, bus4, p_mw=10, controllable=False)

#create generators
eg = create_ext_grid(net, bus1)
g0 = create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80,  vm_pu=1.01, controllable=True)
g1 = create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.01, controllable=True)

In [ ]:
create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=-1)
create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=-1)
create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=0)
runopp(net, verbose=True)

Because of the negative costs, the OPF now maximizes power generation at the generators, which is constrained by their maximum power:

In [ ]:
pd.concat([net.res_gen.p_mw, net.gen.min_p_mw, net.gen.max_p_mw], axis=1)

While gen 1 is operating at the limit, gen 0 is below the maximum output. Apparently the generator can not reach its maximum output without violating at least one power flow constraint. Let's check on the constraints.

The line and transformer constraints are not reached:

In [ ]:
pd.concat([net.line.max_loading_percent, net.res_line.loading_percent], axis=1)

In [ ]:
pd.concat([net.trafo.max_loading_percent, net.res_trafo.loading_percent], axis=1)

But the voltage constraints are:

In [ ]:
pd.concat([net.res_bus.vm_pu, net.bus.min_vm_pu, net.bus.max_vm_pu], axis=1)

Obviously the voltage profile was the limiting factor for the generator feed-in. If we relax this constraint a little bit:

In [ ]:
net.bus["max_vm_pu"] = 1.05
runopp(net)

We see an increased feed-in of the generators:

In [ ]:
pd.concat([net.res_gen.p_mw, net.gen.min_p_mw, net.gen.max_p_mw], axis=1)

In [ ]:
net.res_bus